# interactive

hvPlot isn't only a plotting library, it is dedicated to make data exploration easier. In this guide you will see how it can help you to get better control over your data pipelines. We define a *data pipeline* as a series of commands that *transform* some data, such as aggregating, filtering, reshaping, renaming, etc. A data pipeline may include a *load* step that will provide the input data to the pipeline, e.g. reading the data from a data base. 

When you analyze some data in a notebook that is for instance held in a Pandas DataFrame, you may find yourself having to re-run many cells after changing the parameters you provide to Pandas' methods, either to get more insights on the data or fine tune an algorithm. `.interactive()` is a solution to improve this rather cumbersome workflow, by which you replace the constant parameters in the pipeline by widgets (e.g. a number slider), that will automatically get displayed next to your pipeline output and will trigger an output update on changes. With this approach all your pipeline parameters are available in one place and you get full interactive control over the pipeline.

`.interactive()` doesn't only work with DataFrames but also with Xarray data structures, this is what we are going to show in this guide. First we will import `hvplot.xarray` which is going make available the `.interactive()` accessor on Xarray objects.

In [ ]:
import hvplot.xarray  # noqa
import xarray as xr

We load a dataset and get a handle on its unique *air* variable.

In [ ]:
ds = xr.tutorial.load_dataset('air_temperature')
air = ds.air
ds

We want to better understand the temporal evolution of the air temperature over different latitudes compared to a baseline. The data pipeline we build includes:

1. filtering the data at one latitude
2. cleaning up the data
3. aggregating the temperatures by time
4. computing a rolling mean
5. subtracting a baseline from the above

The output we choose for now is the `.describe()` method of Pandas. This pipeline has two parameters, the latitude and the temporal window of the rolling operation.

In [ ]:
LATITUDE = 30.
ROLLING_WINDOW = '1D'

baseline = air.sel(lat=LATITUDE).mean().item()
pipeline = (
    air
    .sel(lat=LATITUDE)
    .to_dataframe()
    .drop(columns='lat')
    .groupby('time').mean()
    .rolling(ROLLING_WINDOW).mean()
    - baseline
)
pipeline.describe()

Without `.interactive()` we would manually change the values of `LATITUDE` and `ROLLING_WINDOW` to see how they affect the pipeline output. Instead we create two widgets with the values we expect them to take, we are basically declaring beforehand our parameter space. To create widgets we import [Panel](https://panel.holoviz.org) and pick two appropriate widgets from its [Reference Gallery](https://panel.holoviz.org/reference/index.html#widgets).

In [ ]:
import panel as pn

w_latitude = pn.widgets.DiscreteSlider(name='Latitude', options=list(air.lat.values))
w_rolling_window = pn.widgets.RadioButtonGroup(name='Rolling window', options=['1D', '7D', '30D'])

Now we instantiate an *Interactive* object by calling `.interactive()` on our data. This object mirrors the underlying object API, it accepts all of its natural operations. We replace the data by the interactive object in the pipeline, and replace the constant parameters by the widgets we have just created.

In [ ]:
airi = air.interactive()

In [ ]:
baseline = airi.sel(lat=w_latitude).mean().item()
pipeline = (
    airi
    .sel(lat=w_latitude)
    .to_dataframe()
    .drop(columns='lat')
    .groupby('time').mean()
    .rolling(w_rolling_window).mean()
    - baseline
)
pipeline.describe()

You can see that now the pipeline when rendered doesn't only consist of its output, it also includes the widgets that control it. Change the widgets' values and observe how the output dynamically updates.

You can notice that `.interactive()` supports the fact that the data type changed in the pipeline (see the call to `.to_dataframe`) and that it also supports math operators (`- baseline`).

A plot would be a better output for this pipeline. We will use `.hvplot()` to create an interactive Bokeh line plot, note that using Pandas' `.plot()` is also possible.

In [ ]:
import hvplot.pandas  # noqa

pipeline.hvplot()

For information on using `.interactive()` take a look at the [User Guide](../user_guide/Interactive.ipynb).